<a href="https://colab.research.google.com/github/Huxwell/ColabNNs/blob/main/cats_n_dogs_create_hybrids_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# import torch.nn as nn
# import torch
# import torch.optim as optim
# import torch.nn.functional as F
# from torchvision import datasets, models, transforms
# from torch.utils.data import DataLoader, Dataset
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import os
import zipfile
import glob
# from tqdm import tqdm
# from PIL import Image
import random # do random.seed(13) before every shuffle. order of shuffle() execution changes results order.

In [13]:
dataset_dir = '/content/drive/MyDrive/cats_n_dogs_unsure/dogs-vs-cats-redux-kernels-edition'
os.listdir(dataset_dir)

['sample_submission.csv', 'test.zip', 'train.zip']

In [14]:
os.makedirs('local_data/', exist_ok=True)
train_dir = 'local_data/train'

In [15]:
with zipfile.ZipFile(os.path.join(dataset_dir, 'train.zip')) as train_zip:
    train_zip.extractall('local_data')

In [16]:
os.listdir(train_dir)[:5]

['cat.2832.jpg',
 'cat.11210.jpg',
 'cat.10450.jpg',
 'dog.5903.jpg',
 'cat.10237.jpg']

In [17]:
cats_list = sorted(glob.glob(os.path.join(train_dir,'cat*.jpg')))
dogs_list = sorted(glob.glob(os.path.join(train_dir,'dog*.jpg')))
print(len(cats_list))
print(len(dogs_list))
random.seed(13)
random.shuffle(cats_list)
random.seed(13) #multiple seed() executions are on purpose.
random.shuffle(dogs_list)
print(cats_list[:3])
print(dogs_list[:3])
# some sanity check to make sure no uncontrolled randomness beyond this point
assert cats_list[2] == 'local_data/train/cat.801.jpg'
assert dogs_list[2] == 'local_data/train/dog.801.jpg'
assert len(cats_list) == len(dogs_list)


12500
12500
['local_data/train/cat.8473.jpg', 'local_data/train/cat.3808.jpg', 'local_data/train/cat.801.jpg']
['local_data/train/dog.8473.jpg', 'local_data/train/dog.3808.jpg', 'local_data/train/dog.801.jpg']
